In [121]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from collections import Counter

# give meta data about book and path
book_path = 'Potter1_de'
language = 'de'
titel = 'Harry Potter und der Stein der Weisen'
author = 'Joanne K. Rowling'
names =['Harry Potter', 'Dursley', 'Hermine Granger', 'Ron Weasley']


# read content of book
book_raw = pd.read_csv("../data/raw/" + book_path + "_raw.csv")

# copy raw content as a list to fist column second row   
book = book_raw.copy()

In [122]:
## layout of book = [{'raw':[], 'meta': {}, 'word_list': [], 'word_bag': {}, '2-gram': [], '3-gram': []}] in seperate csv saved at the end of the notebook
# Note - per Hand entfert: , " da Probleme mit pd.read_csv
# Note - per Hand bereinigt: Inhaltsverzeichnis, erste letzte Seite, Kopf und Fußzeilen

In [123]:
## list of contractions
contractions = {}

if language == 'en':
    contractions = {
"ain't": "are not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "I would",
"i'd've": "I would have",
"i'll": "I will",
"i'll've": "I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [124]:
## tidy book

# convert everything to string
book = book.applymap(str)           # it should be string anyway

# save concat all rows in book to get a very long sentence without line breaks
sentence = ''.join(book['content'].tolist())

# remove: numbers
sentence = ''.join([i for i in sentence if not i.isdigit()])

# add spaces before every capitol char
sentence = re.sub('[A-Z]', lambda pat: ' '+pat.group(0), sentence)

# de - convert 's to es
if language == 'de':
    sentence = re.sub("(\S+)(\'s)", lambda pat: pat.group(1)+" es", sentence)
    sentence = re.sub("(\S+)(\'n)", lambda pat: pat.group(1)+"en", sentence)
    sentence = re.sub("(\S+)(\')", lambda pat: pat.group(1)+"e", sentence)

# lower case everyting
sentence = sentence.lower()

# replace contractions (en)
sentence = re.sub('\n’\n', '\'', sentence)
sentence = re.sub('’\n', '\'', sentence)
sentence = re.sub('’', '\'', sentence)
for word in sentence.split():
    if word in contractions:
        sentence = re.sub(word, contractions[word], sentence)

# remove . : ! ? " - ;  etc pp w regex
sentence = re.sub('--', ' ', sentence)
sentence = re.sub('-', '', sentence)
sentence = re.sub('β', 'ß', sentence)
sentence = re.sub('ﬁ', 'fi', sentence)
sentence = re.sub('[^0-9a-zA-ZäüöÄÖÜß ]+', ' ', sentence)

#print(sentence)

In [125]:
## book_list
book_lst = list(sentence.split())
alnumcheck = [book_lst[i].isalnum() for i in range(len(book_lst))]
book_list = pd.DataFrame(book_lst)
book_list = book_list.rename(columns={0: "list"})

In [126]:
## book_bag
book_bag = {}

for e in book_lst:
    if e in book_bag:
        book_bag[e] += 1
    else:
        book_bag.update({e: 1})

book_bag = pd.DataFrame(book_bag.items(), columns=['word','freq'])

In [127]:
## book_meta: titel, author, pages, number of all words, number of unique words, names (list), 
book_meta = {}

book_meta.update({'titel': titel})
book_meta.update({'author': author})
book_meta.update({'pages': [int(s) for s in re.findall(r'\b\d+\b', book.content[len(book['content'])-1])][-1]})
book_meta.update({'num_words': len(book_list['list'])})
book_meta.update({'num_uniq': book_list['list'].nunique()})
book_meta.update({'names': names})
book_meta.update({'language': language})

book_meta = pd.DataFrame(book_meta.items(), columns=['prop','value'])

In [128]:
## book_2gram
bigram_bag = Counter(zip(book_lst, book_lst[1:])).most_common()
bigram_list = [list(e) for e in zip(book_lst, book_lst[1:])]

book_2gramBag = pd.DataFrame(bigram_bag, columns=['bigram','freq'])
book_2gramList = pd.DataFrame(bigram_list, columns=['first','second'])

In [129]:
## book_3gram
def find_ngrams(input_list, n):
  return zip(*(input_list[i:] for i in range(n)))

trigram_bag = Counter(find_ngrams(book_lst, 3)).most_common()
trigram_list = [list(e) for e in find_ngrams(book_lst, 3)]

book_3gramBag = pd.DataFrame(trigram_bag, columns=['trigram','freq'])
book_3gramList = pd.DataFrame(trigram_list, columns=['first','second', 'third'])


In [130]:
## book_5gram
def find_ngrams(input_list, n):
  return zip(*(input_list[i:] for i in range(n)))

quadrigram_bag = Counter(find_ngrams(book_lst, 4)).most_common()
quadrigram_list = [list(e) for e in find_ngrams(book_lst, 4)]

book_4gramBag = pd.DataFrame(quadrigram_bag, columns=['quadrigram','freq'])
book_4gramList = pd.DataFrame(quadrigram_list, columns=['first','second', 'third', 'fourth'])

In [131]:
## book_001gram
def find_ngrams(input_list, n):
  return zip(*(input_list[i:] for i in range(n)))

per = len(book_lst) // 100 
onepergram_bag = Counter(find_ngrams(book_lst, per)).most_common()
onepergram_list = [list(e) for e in find_ngrams(book_lst, per)]

# book_001gramBag = pd.DataFrame(onepergram_bag, columns=['onepergram','freq'])
book_001gramList = pd.DataFrame(onepergram_list)

In [132]:
book_list.to_csv("../data/processed/" + book_path + "_list.csv", index = False)
book_bag.to_csv("../data/processed/" + book_path + "_bag.csv", index = False)
book_meta.to_csv("../data/processed/" + book_path + "_meta.csv", index = False)
book_2gramList.to_csv("../data/processed/" + book_path + "_2gramList.csv", index = False)
book_2gramBag.to_csv("../data/processed/" + book_path + "_2gramBag.csv", index = False)
book_3gramList.to_csv("../data/processed/" + book_path + "_3gramList.csv", index = False)
book_3gramBag.to_csv("../data/processed/" + book_path + "_3gramBag.csv", index = False)
book_4gramList.to_csv("../data/processed/" + book_path + "_4gramList.csv", index = False)
book_4gramBag.to_csv("../data/processed/" + book_path + "_4gramBag.csv", index = False)
book_001gramList.to_csv("../data/processed/" + book_path + "_001gramList.csv", index = False)
# book_001gramBag.to_csv("../data/processed/" + book_path + "_001gramBag.csv", index = False)